In [2]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

In [5]:
'''
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = '/usr/share/fonts/NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()
'''

'\nimport matplotlib\nimport matplotlib.pyplot as plt\nimport matplotlib.font_manager as fm\nfont_path = \'/usr/share/fonts/NanumGothic.ttf\'\nfont_name = fm.FontProperties(fname=font_path, size=10).get_name()\nplt.rc(\'font\', family=font_name, size=12)\nplt.rcParams["figure.figsize"] = (20, 10)\nregister_matplotlib_converters()\n'

In [4]:
directory = './res/'

In [5]:
# Magazine. json

magazine = pd.read_json(directory + 'magazine.json', lines=True)

In [6]:
magazine.shape

(27967, 2)

In [6]:
magazine.head()

,id,magazine_tag_list
0,38842,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]"
1,11540,"[tea, food]"
2,11541,[food]
3,11546,"[브런치북, 일상, 시, 사람]"
4,11544,"[감성에세이, 노래, 음악에세이]"


In [7]:
print(type(magazine))

<class 'pandas.core.frame.DataFrame'>


In [8]:
# Metadata.json

metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [9]:
metadata.shape

(643104, 9)

In [10]:
metadata.head()

,article_id,display_url,id,keyword_list,magazine_id,reg_ts,sub_title,title,user_id
0,782,https://brunch.co.kr/@bookdb/782,@bookdb_782,"[여행, 호주, 국립공원]",8982,1474944427000,세상 어디에도 없는 호주 Top 10,"사진으로 옮기기에도 아까운, 리치필드 국립공원",@bookdb
1,81,https://brunch.co.kr/@kohwang56/81,@kohwang56_81,"[목련꽃, 아지랑이, 동행]",12081,1463092749000,,[시] 서러운 봄,@kohwang56
2,4,https://brunch.co.kr/@hannahajink/4,@hannahajink_4,[],0,1447997287000,무엇 때문에,무엇을 위해,@hannahajink
3,88,https://brunch.co.kr/@bryceandjuli/88,@bryceandjuli_88,"[감정, 마음, 위로]",16315,1491055161000,,싫다,@bryceandjuli
4,34,https://brunch.co.kr/@mijeongpark/34,@mijeongpark_34,"[유럽여행, 더블린, 아일랜드]",29363,1523292942000,#7. 내 친구의 집은 어디인가,Dubliner#7,@mijeongpark


In [11]:
# users.json

users = pd.read_json(directory + 'users.json', lines=True)

In [12]:
users.shape

(310758, 3)

In [13]:
users.head()

,following_list,id,keyword_list
0,"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a,[]
1,"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed,[]
2,"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5,[]
3,"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d,[]
4,"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62,[]


In [7]:
# Read Files

read_file_list = glob.glob(directory + 'read/*')
exclude_file_lst = ['read.tar']

read_df_lst = []
for f in read_file_list :
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst :
        print(file_name)
    else :
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [8]:
read = pd.concat(read_df_lst)

In [9]:
read.shape

(3507097, 5)

In [10]:
read.head()

,raw,dt,hr,user_id,article_id
0,#1f756fba1b5058dcedc2d70e630de726 @jidesign_55...,20181120,14,#1f756fba1b5058dcedc2d70e630de726,@jidesign_55 @jidesign_57 @jidesign_25
1,#0e04a43eb63b9988a5a5c3a6c5a0ff79 @peregrino97...,20181120,14,#0e04a43eb63b9988a5a5c3a6c5a0ff79,@peregrino97_779 @noey_130
2,#c3c266cbe2a759423114f2f250ccc02c @jangsypd_95...,20181120,14,#c3c266cbe2a759423114f2f250ccc02c,@jangsypd_95 @jangsypd_95
3,#6a79073d959f42de59709c772729bf15 @jordan777_2...,20181120,14,#6a79073d959f42de59709c772729bf15,@jordan777_2417 @jordan777_2417 @jordan777_241...
4,#725721587e7b6934b1917fe722f69914 @begini_440 ...,20181120,14,#725721587e7b6934b1917fe722f69914,@begini_440 @begini_441


In [11]:
# 탐색하기 좋은 데이터 포맷으로 변경
def chainer(s) :
    return list(chain.from_iterable(s.str.split(' ')))

In [13]:
read_cnt_by_user = read['article_id'].str.split(' ').map(len)



In [15]:
read_cnt_by_user.head()
read_cnt_by_user.shape

(3507097,)

In [ ]:
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])
                        })

In [ ]:
read_raw.shape

In [ ]:
read_raw.head()